# DyNet RNNs チュートリアル

チュートリアルを意訳しています。

参考
* https://github.com/clab/dynet/blob/master/examples/python/tutorials/RNNs.ipynb
* https://media.readthedocs.org/pdf/dynet/latest/dynet.pdf

In [1]:
from dynet import *

# LSTM/RNN 全体像

１層のRNNは隠れ層を次のように分けている $h_1,...,h_k$, $h_i$ は時間ごとの指標を表している
それぞれの層$h_i$は入力$x_i$と出力$r_i$に加えて過去の入力$r_{i-1}$も与える
複数層の場合は連続ではなく網目状になる。例えば

* $h_1^3,...,h_k^3$
* $h_1^2,...,h_k^2$
* $h_1^1,...h_k^1$

上記の場合は

隠れ層$h_i^1$の入力は$x_i$と$r_{i-1}^1$

隠れ層$h_i^2$の入力は$r_i^1$と$r_{i-1}^2$　など

# LSTM インターフェース

RNN / LSTM / GRU は同様のインターフェースになる。
"builder"が連続列のためのパラメータを作成する

In [2]:
model = Model()
NUM_LAYERS = 2
INPUT_DIM = 50
HIDDEN_DIM = 10
builder = LSTMBuilder(NUM_LAYERS, INPUT_DIM, HIDDEN_DIM, model)

"Builder"でRNNの内部パラメータをモデルに加えることでパラメータを意識しなくて良くなるが最適化の際は同時に行ってくれる！！

In [3]:
s0 = builder.initial_state()
s0

In [4]:
print(vecInput(10))
x1 = vecInput(INPUT_DIM)
print(x1)

expression 22/0
expression 23/0


In [5]:
s1 = s0.add_input(x1)
y1 = s1.output()

In [6]:
y1.npvalue().shape

(10,)

In [7]:
s1.h()

(expression 33/0, expression 43/0)

In [8]:
s2 = s1.add_input(x1)
y2 = s2.output()
y2

expression 67/0

１層の層で$y2$は隠れ層の状態と等しい。しかしながら最終層でもある。
隠れ層の状態にアクセスしたい場合は下記のメソッドを使用する

In [9]:
s2.h()

(expression 55/0, expression 67/0)

LSTMの状態の出力を比較してみると

In [10]:
print("all layers:", s1.h())

all layers: (expression 33/0, expression 43/0)


In [11]:
print( s1.s())

(expression 29/0, expression 39/0, expression 33/0, expression 43/0)


同様のインターフェースでシンプルなRNNも作成可能

In [12]:
rnnbuilder = SimpleRNNBuilder(NUM_LAYERS, INPUT_DIM, HIDDEN_DIM, model)

rs0 = rnnbuilder.initial_state()

rs1 = rs0.add_input(x1)
ry1 = rs1.output()
print( rs1.s())

(expression 75/0, expression 77/0)


* 出力は"output"メソッドで行う
* 隠れ層の値は"h()"メソッドで行う
* 状態は"s()"で確認する

LSTMのみ出力が４つになっている。これは入力層、忘却層、過去の入力層、出力層が存在するためである。

In [13]:
rnn_h = rs1.h()
rnn_s = rs1.s()
print("RNN h:", rnn_h)
print("RNN s:", rnn_s)

lstm_h = s1.h()
lstm_s = s1.s()
print("LSTM h:", lstm_h)
print("LSTM s:", lstm_s)

RNN h: (expression 75/0, expression 77/0)
RNN s: (expression 75/0, expression 77/0)
LSTM h: (expression 33/0, expression 43/0)
LSTM s: (expression 29/0, expression 39/0, expression 33/0, expression 43/0)


# RNNとLSTMインターフェースのオプション

Stack LSTM: 過去の状態を覚えておけるのでそこに新たに追加してグラフの作成が行える。



In [14]:
s2 = s1.add_input(x1)
s3 = s2.add_input(x1)
s4 = s3.add_input(x1)

# s3に新しい入力を加える
s5 = s3.add_input(x1)

#下記の２つの異なる列ができている
# s0, s1, s2, s3, s4
# s0, s1, s2, s3, s5

assert(s5.prev() == s3)
assert(s4.prev() == s3)

s6 = s3.prev().add_input(x1)
# s0, s1, s2, s6

In [15]:
s6.h()

(expression 185/0, expression 197/0)

4つの列が出力される

In [16]:
s6.s()

(expression 181/0, expression 193/0, expression 185/0, expression 197/0)

# メモリの効果的な変換

入力列をlist化して今までのことを一気にやってくれる手法があります。

In [17]:
state = rnnbuilder.initial_state()
xs = [x1, x1, x1]
states = state.add_inputs(xs)
outputs = [s.output() for s in states]
hs = [s.h() for s in states]
print(outputs)
print(hs)

[expression 201/0, expression 205/0, expression 209/0]
[(expression 199/0, expression 201/0), (expression 203/0, expression 205/0), (expression 207/0, expression 209/0)]


transduceを使用すればもっとメモリを効率よく使用でき、少ない行数でかけます。

In [18]:
state = rnnbuilder.initial_state()
xs = [x1, x1, x1]
outputs = state.transduce(xs)
print(outputs)

[expression 213/0, expression 217/0, expression 221/0]


# 言葉レベルのLSTM

ここから本格的な応用に入ります。言葉レベルのRNNですprint(outputs)

* モデルのパラメータの初期的設定
* 言葉の設定。
* 明示的に終了と始まり符号を表す<EOS>を追加
* コンピューターで扱うため、数字と文字の変換用の辞書を用意

In [19]:
import random
from collections import defaultdict
from itertools import count
import sys

LAYERS = 2
INPUT_DIM = 50 
HIDDEN_DIM = 50  

characters = list("abcdefghijklmnopqrstuvwxyz ")
characters.append("<EOS>")

int2char = list(characters)
char2int = {c:i for i,c in enumerate(characters)}

VOCAB_SIZE = len(characters)

# モデルの設定

* modelのパラメータ設定
* 61ページにlookupパラメータの意味があります。Word Embeddingのためです。
http://phontron.com/slides/emnlp2016-dynet-tutorial-part1.pdf
* params["lookup"]は入力用
* params["R"]は出力用
* params["bias"]はバイアス用

In [20]:
model = Model()


srnn = SimpleRNNBuilder(LAYERS, INPUT_DIM, HIDDEN_DIM, model)
lstm = LSTMBuilder(LAYERS, INPUT_DIM, HIDDEN_DIM, model)

params = {}
params["lookup"] = model.add_lookup_parameters((VOCAB_SIZE, INPUT_DIM))
params["R"] = model.add_parameters((VOCAB_SIZE, HIDDEN_DIM))
params["bias"] = model.add_parameters((VOCAB_SIZE))

* 一つの文章に対してのメソッド：文章ごと長さが異なり処理を分けたいため定義
* pick関数は資料の38ページ参照
https://media.readthedocs.org/pdf/dynet/latest/dynet.pdf

In [21]:
# return compute loss of RNN for one sentence
def do_one_sentence(rnn, sentence):
    # setup the sentence
    renew_cg()
    s0 = rnn.initial_state()
    
    
    R = parameter(params["R"])
    bias = parameter(params["bias"])
    lookup = params["lookup"]
    sentence = ["<EOS>"] + list(sentence) + ["<EOS>"]
    sentence = [char2int[c] for c in sentence]
    s = s0
    loss = []
    for char,next_char in zip(sentence,sentence[1:]):
        s = s.add_input(lookup[char])
        probs = softmax(R*s.output() + bias)
        loss.append( -log(pick(probs,next_char)) )
    loss = esum(loss)
    return loss

モデルからの文字列生成

In [22]:
# generate from model:
def generate(rnn):
    def sample(probs):
        rnd = random.random()
        for i,p in enumerate(probs):
            rnd -= p
            if rnd <= 0: break
        return i
    
    # setup the sentence
    renew_cg()
    s0 = rnn.initial_state()
    
    R = parameter(params["R"])
    bias = parameter(params["bias"])
    lookup = params["lookup"]
    
    s = s0.add_input(lookup[char2int["<EOS>"]])
    out=[]
    while True:
        probs = softmax(R*s.output() + bias)
        probs = probs.vec_value()
        next_char = sample(probs)
        out.append(int2char[next_char])
        if out[-1] == "<EOS>": break
        s = s.add_input(lookup[next_char])
    return "".join(out[:-1]) # strip the <EOS>

学習のための関数

In [23]:
def train(rnn, sentence):
    trainer = SimpleSGDTrainer(model)
    for i in range(200):
        loss = do_one_sentence(rnn, sentence)
        loss_value = loss.value()
        loss.backward()
        trainer.update()
        if i % 5 == 0:
            print(loss_value)
            print(generate(rnn))

#実行例 RNN

In [24]:
sentence = "a quick brown fox jumped over the lazy dog"
train(srnn, sentence)

162.6923828125
ijirerdeo
94.03389739990234
lqazd hjohkbeyinoo qhbokdr m zp xxe qeg
51.72581481933594
fo udxm dxe jzbd q
29.075246810913086
jlsun daqu umidvg
10.539573669433594
yllzw tfmr tue lagyv ky lownowmi znefoqujck  weunvtp z
3.530012607574463
a quick brown fox jumped over the laz
1.117006778717041
a luick broqn fox jumpkd over the lazy dog
0.6907472014427185
a qupck brown fox jumked over the lazy dog
0.5008178353309631
a quick brown fox jumped over the lazy dog
0.39229434728622437
a quick brown fox jumped over the lazy dog
0.3220541477203369
a quick brown fox jumped over the lazy dog
0.2728939652442932
a quick brown fox jumped over the lazy dog
0.2365807443857193
a quick brown fox jumped over the lazy dog
0.20867912471294403
a quick brown fox jumped over the lazy dog
0.18658441305160522
a quick brown fox jumped over the lazy dog
0.1686524897813797
a quick brown fox jumped over the lazy dog
0.1538228988647461
a quick brown fox jumped over the lazy dog
0.14135073125362396
a quick b

#実行例 LSTM

In [25]:
sentence = "a quick brown fox jumped over the lazy dog"
train(lstm, sentence)

143.7174530029297
bo 
130.62815856933594
ve zdohact oajgok 
123.93404388427734
lnowegj wfteau  di okoe do wdocfpmdub  t ohwlbuetguvuogdks
115.06339263916016
ne
101.93220520019531
k qii odxm oexyoo dxfo kmggdvednpvufeze d adugn coxod xpow xe xr wyp ocy nuojg oentn noqd ho
91.08451843261719
ofn igr woo bjbxd ne xed j
75.37924194335938
 uu huee lap aayw ugop exr kqcr coyz  or
61.78849792480469
g ocrt rbx rown mjmred orenr fbwddoro of
50.449432373046875
n upke ove hzvdb
40.671566009521484
 qnc ijk rwn oown wfi ummed oah lau uzhcr bo
32.17814254760742
f qukg
22.785524368286133
aq uizdk brwrnn fuoc hra  oime dorr tve lzrz dog
17.198972702026367
qa quijped ooher lhlek qahy oogx over the ltaz ogb
12.214446067810059
fy quicc baokf fox jjck brow fox mumep oae zced ao
8.37203598022461
p  jhk boww fox juce over the tay dog
4.8374786376953125
a quick brownff ox juppp
2.964689254760742
a quick brown fox jumped over the lazy dug
1.7102702856063843
a qzick brown fox jumped over the lazy dog
1.28169977